In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load your dataset
def load_data(dataset_path, img_size=(48, 48)):
    images = []
    labels = []
    classes = os.listdir(dataset_path)
    print(type(classes))
    print(classes)
    classes.pop(1)
    class_map = {class_name: i for i, class_name in enumerate(classes)}
    
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, img_size)  # Resize
            images.append(img)
            labels.append(class_map[class_name])

    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1) / 255.0  # Normalize
    labels = to_categorical(np.array(labels), num_classes=len(classes))  # One-hot encode labels
    return images, labels, class_map

# Load data from your dataset
dataset_path = "Documents/facial_expression/train"
X, y, class_map = load_data(dataset_path)

# Split dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(class_map), activation='softmax')  # Output neurons match class count
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Save model
model.save('facial_expression_model.h5')

# Real-time facial expression recognition
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = np.expand_dims(roi_gray, axis=0)
        roi_gray = np.expand_dims(roi_gray, axis=-1) / 255.0

        prediction = model.predict(roi_gray)
        expression = np.argmax(prediction)

        expressions = list(class_map.keys())  # Get class labels
        label = expressions[expression]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow('Facial Expression Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


<class 'list'>
['happy', '.DS_Store', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust']


/opt/anaconda3/envs/example/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 106s 117ms/step - accuracy: 0.2761 - loss: 1.7608 - val_accuracy: 0.4277 - val_loss: 1.4545
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 101s 113ms/step - accuracy: 0.4544 - loss: 1.4225 - val_accuracy: 0.4980 - val_loss: 1.3341
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 124s 138ms/step - accuracy: 0.5083 - loss: 1.2908 - val_accuracy: 0.5358 - val_loss: 1.2192
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 101s 113ms/step - accuracy: 0.5485 - loss: 1.1843 - val_accuracy: 0.5546 - val_loss: 1.1911
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 98s 110ms/step - accuracy: 0.5861 - loss: 1.1027 - val_accuracy: 0.5678 - val_loss: 1.1646
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 150s 168ms/step - accuracy: 0.6164 - loss: 1.0152 - val_accuracy: 0.5691 - val_loss: 1.1454
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 98s 109ms/step - accuracy: 0.6403 - loss: 0.9520 - val_accuracy: 0.5717 - val_loss: 1.1567
Epoch 8/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 111s 124ms/step - accuracy: 0.6698 - l

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━

In [1]:
from keras.models import load_model
import cv2
import numpy as np
import os
import json

# Load the trained model
model = load_model('facial_expression_model.h5')
predictions_list = []
# Load class labels (Ensure the same class_map is used as during training)
class_map = {0: "Happy", 1: "sad", 2: "fear", 3: "surprise", 4: "Neutral",5:"Angry",6:"disgust"}  # Replace with your actual class labels
#{'happy': 0, 'sad': 1, 'fear': 2, 'surprise': 3, 'neutral': 4, 'angry': 5, 'disgust': 6}

# Real-time facial expression recognition
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = np.expand_dims(roi_gray, axis=0)
        roi_gray = np.expand_dims(roi_gray, axis=-1) / 255.0

        prediction = model.predict(roi_gray)
        expression = np.argmax(prediction)
        predictions_list.append(class_map[expression])  # Store class name

        label = class_map[expression]  # Get the predicted class label

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow('Facial Expression Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
with open("predictions.json", "w") as f:
    json.dump(predictions_list, f)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━

In [2]:
print(predictions_list)

['Neutral', 'sad', 'Angry', 'fear', 'sad', 'Happy', 'Angry', 'sad', 'Neutral', 'Neutral', 'Neutral', 'Happy', 'Happy', 'Happy', 'Angry', 'Happy', 'Angry', 'Happy', 'Happy', 'Angry', 'Happy', 'Happy', 'Happy', 'Happy', 'Angry', 'Neutral', 'Neutral', 'Angry', 'Neutral', 'Angry', 'Angry', 'Angry', 'fear', 'Neutral', 'fear', 'Neutral', 'Angry', 'sad', 'fear', 'Angry', 'fear', 'fear', 'fear', 'Neutral', 'Neutral', 'Happy', 'sad', 'sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Neutral', 'Neutral', 'Neutral', 'sad', 'sad', 'Happy', 'sad', 'sad', 'Neutral', 'sad', 'sad', 'sad', 'Neutral', 'sad', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'sad', 'sad', 'Neutral', 'Neutral', 'sad', 'Neutral', 'Happy', 'sad', 'sad', 'Happy', 'sad', 'Happy', 'sad', 'Happy', 'sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'sad', 'sad', 'sad', 'sad', 'Happy', 'Happy', 'Neutral', 'Happy', 'Happy', 'Neutral', 

In [3]:
print(predictions_list)
import json
from collections import Counter
with open("predictions.json", "w") as f:
    json.dump(predictions_list, f)

frequency_dict = Counter(predictions_list)
print(frequency_dict) 

Counter({'Neutral': 810, 'sad': 457, 'Happy': 282, 'Angry': 232, 'fear': 221, 'surprise': 13, 'disgust': 2})


In [4]:
freq= frequency_dict.values()
max_emo= max(freq)
for keys in frequency_dict:
    if frequency_dict.get(keys) == max_emo:
        print(keys)
        break

Neutral
